In [1]:
import csv
import json
import requests
import time
import copy
data_dir = '/home/ialok/code/la_hack/'
postal_code_dataset = data_dir + 'agences-de-pm-avec-coordonnes-spatiales.csv'
port_data = data_dir + "Dataset/PortByType.geojson"

# URL_KEY = 'Ihw4aanLMir7RjX2w21PWV14Fy2nOHNo'
URL_KEY = 'A4oZrvdr8QBAAUUpxc4bdAeG2IxK9RlJ'

In [2]:
# Return port names from the geoJson files
def return_port_names(ports_file_name):
    with open(ports_file_name, 'r') as fr:
        json_port_data = json.load(fr)
    
    port_names = list()
    for i in json_port_data['features']:
        # print i, "\n\n"
        port_names.append(i['properties']['MAIN_PORT_'])
    return set(port_names)

def return_postal_names(postal_code_dataset):
    with open(postal_code_dataset, 'r') as csv_file:
        postal_data_reader = csv.reader(csv_file, delimiter=',')
        # Skip header
        postal_data_reader.next()
        postal_name = list()
        for row in postal_data_reader:
            postal_name.append(row[2])
    return set(postal_name)

# return_port_names(port_data)
return_postal_names(postal_code_dataset)

{'ASSILAH',
 'IGUIDI',
 'MIKKES',
 'EL KELAA DES SRAGHNA',
 'BENSLIMANE',
 'BAB LAMRISSA',
 'TABIA',
 'OUAD ESSAFA',
 'MOULAY BOUAZZA',
 'ARGANA',
 'TARMIGT',
 'EL KSIBA',
 'EL HAMMAM',
 'IDELSANE',
 'IMRABTEN',
 'AIT OURIBEL',
 'OUKAIMDEN',
 "M'HAJER",
 "M'NABHA",
 'KISSANE',
 'LAANOUSSAR',
 "M'RAMER",
 'AIT BAZZA',
 'KETAMA',
 'TEMARA',
 'SIDI YAHYA OU YOUSSEF',
 'TIGHEDOUINE',
 'HAD OUED IFRANE',
 'KSAR SGHIR',
 'SIDI AAYAD',
 'SIDI ABED',
 'OULAD HCINE',
 'LAQRAQRA',
 'ANERGUI',
 'BZOU',
 'BOUHLOU',
 'FOUM OUDI',
 'GHOUAZI',
 'ZOUAGHA',
 'ABADOU',
 'HAY-HASSANI',
 'AIN TIZGHA',
 "BNI M'HAMED SIJELMASSA",
 'AIT TALEB',
 'BNI BOUZRA',
 "AGHBALOU N'KERDOUS",
 'ASSOUKHOUR ASSAWDA',
 'MIRLEFT',
 'EL-JADIDA',
 'AIN DFALI',
 'AKARMA',
 'AFALLA ISSEN',
 'RASS OUMLIL',
 'BNI QUOLLA',
 'HARHOURA',
 'HASSI BERKANE',
 'DAR CHAOUI',
 'TAGHZOUT',
 'TROUGOUT',
 'AIT IZDEG',
 'MALLOUSSA',
 'AL MAHBASS',
 'AJDIR',
 'ASSA',
 'SIDI BOUBKER',
 'OURTZAGH',
 'BEN TAIEB',
 'JORF EL MELHA',
 'MELLILA',
 '

In [6]:
# Route(s) between ports and all the cities

class Routes():
    
    def __init__(self, source, key):
        self.source = source
        self.key = key
        
    def dist_map(self, dest):
        """
            Find the best route between the port(s)
            and the destination(post office(s))
        """
        url = 'http://open.mapquestapi.com/directions/v2/route'
        payload = {'from': self.source, 'to': dest, 'key': self.key}
        response = requests.get(url, params=payload)

        if response.status_code != 200:
            print "FAIL: ", response.url
            raise Exception("URL failed")
        else:
            dist_matrix = {}
            route = list()
            print "SUCCESS: ", response.url

            json_text = response.json()
            # Add start and end
            route.append([json_text['route']['locations'][0]['latLng']['lng'], 
                          json_text['route']['locations'][0]['latLng']['lat']
                         ])                                             
            for idx, row in enumerate(json_text['route']['legs'][0]['maneuvers']):
                route.append([row['startPoint']['lng'],
                              row['startPoint']['lat']
                              ])
            try:
                route.append([json_text['route']['locations'][1]['latLng']['lng'], 
                              json_text['route']['locations'][1]['latLng']['lat']
                             ]) 
            except:
                pass
            # print route
            dist_matrix['route'] = copy.deepcopy(route)
            dist_matrix['destination'] = dest
            dist_matrix['source'] = self.source
        
            # json_file = json.loads(dist_matrix)
            return json.dumps(dist_matrix)
    
    
    def get_source(self):
        return self.source
    


# http://open.mapquestapi.com/directions/v2/route?key=Ihw4aanLMir7RjX2w21PWV14Fy2nOHNo&from=FES-MEDINA&to=CHARF-MGHOGHA

routes = Routes('FES-MEDINA', URL_KEY)
path = routes.dist_map('CHARF-MGHOGHA')
print path
# print routes.get_dist_map()
# routes = find_route('FES-MEDINA', 'CHARF-MGHOGHA', URL_KEY)
# print routes

SUCCESS:  http://open.mapquestapi.com/directions/v2/route?to=CHARF-MGHOGHA&from=FES-MEDINA&key=A4oZrvdr8QBAAUUpxc4bdAeG2IxK9RlJ
{"source": "FES-MEDINA", "route": [[-4.9738, 34.063119], [-4.968237, 34.060466], [-4.968472, 34.058895], [-4.966259, 34.059299], [-4.994145, 34.048042], [-4.996763, 34.044929], [-5.003039, 34.037399], [-5.008255, 34.031688], [-5.01119, 34.028354], [-5.014245, 34.02634], [-5.015801, 34.023986], [-5.01794, 34.019653], [-5.018721, 34.017616], [-5.031665, 33.965476], [-5.031619, 33.965457], [-5.035591, 33.955474], [-5.035808, 33.955348], [-6.550344, 34.015716], [-6.706041, 34.02819], [-6.712046, 34.031192], [-5.930292, 35.676563], [-5.928146, 35.677688], [-5.927453, 35.67807], [-5.924588, 35.679744], [-5.921393, 35.681385], [-5.882939, 35.728343], [-5.84137, 35.747592], [-5.803906, 35.769527], [-5.799108, 35.777492], [-5.793641, 35.775516], [-5.767454, 35.782073], [-5.765484, 35.77996], [-5.762542, 35.776897], [-5.763517, 35.777192]], "destination": "CHARF-MGHOGHA

In [47]:
json.dumps(path)

'{"source": "FES-MEDINA", "route": [[-4.9738, 34.063119], [-4.968237, 34.060466], [-4.968472, 34.058895], [-4.966259, 34.059299], [-4.994145, 34.048042], [-4.996763, 34.044929], [-5.003039, 34.037399], [-5.008255, 34.031688], [-5.01119, 34.028354], [-5.014245, 34.02634], [-5.015801, 34.023986], [-5.01794, 34.019653], [-5.018721, 34.017616], [-5.031665, 33.965476], [-5.031619, 33.965457], [-5.035591, 33.955474], [-5.035808, 33.955348], [-6.550344, 34.015716], [-6.706041, 34.02819], [-6.712046, 34.031192], [-5.930292, 35.676563], [-5.928146, 35.677688], [-5.927453, 35.67807], [-5.924588, 35.679744], [-5.921393, 35.681385], [-5.882939, 35.728343], [-5.84137, 35.747592], [-5.803906, 35.769527], [-5.799108, 35.777492], [-5.793641, 35.775516], [-5.767454, 35.782073], [-5.765484, 35.77996], [-5.762542, 35.776897], [-5.763517, 35.777192]], "destination": "CHARF-MGHOGHA"}'

In [ ]:
port_names = return_port_names(port_data)
postal_names = return_postal_names(postal_code_dataset)

# iterate through the ports
for port in port_names:    
    # iterate over the postal codes    
    for dest in postal_names:
        route = Routes(port, URL_KEY)
        print port, "-", dest        
        try:
            # print route.get_dist_map()
            with open('port_to_place_2.txt', 'a') as fw:
                fw.write(route.dist_map(dest))
                fw.write("\n")
        except Exception as e:
            # Keep running
            print e
            pass
        time.sleep(0.7)

EL JADIDA - ASSILAH
SUCCESS:  http://open.mapquestapi.com/directions/v2/route?to=ASSILAH&from=EL+JADIDA&key=A4oZrvdr8QBAAUUpxc4bdAeG2IxK9RlJ
EL JADIDA - IGUIDI
SUCCESS:  http://open.mapquestapi.com/directions/v2/route?to=IGUIDI&from=EL+JADIDA&key=A4oZrvdr8QBAAUUpxc4bdAeG2IxK9RlJ
EL JADIDA - MIKKES
SUCCESS:  http://open.mapquestapi.com/directions/v2/route?to=MIKKES&from=EL+JADIDA&key=A4oZrvdr8QBAAUUpxc4bdAeG2IxK9RlJ
EL JADIDA - EL KELAA DES SRAGHNA
SUCCESS:  http://open.mapquestapi.com/directions/v2/route?to=EL+KELAA+DES+SRAGHNA&from=EL+JADIDA&key=A4oZrvdr8QBAAUUpxc4bdAeG2IxK9RlJ
EL JADIDA - BENSLIMANE
SUCCESS:  http://open.mapquestapi.com/directions/v2/route?to=BENSLIMANE&from=EL+JADIDA&key=A4oZrvdr8QBAAUUpxc4bdAeG2IxK9RlJ
EL JADIDA - BAB LAMRISSA
SUCCESS:  http://open.mapquestapi.com/directions/v2/route?to=BAB+LAMRISSA&from=EL+JADIDA&key=A4oZrvdr8QBAAUUpxc4bdAeG2IxK9RlJ
EL JADIDA - TABIA
SUCCESS:  http://open.mapquestapi.com/directions/v2/route?to=TABIA&from=EL+JADIDA&key=A4oZrvdr8QBA

In [ ]:
json_file = json.loads(response.text)
print json_file['route']['locations'][0]['latLng']
print json_file['route']['legs'][0]['maneuvers'][0]['startPoint']
print json_file['route']['locations'][1]['latLng']
# print json_file['route']['legs']

In [27]:
from geojson import LineString, Feature, FeatureCollection, dumps
import json
feature_list = []
with open('port_to_place_2.txt', 'r') as fr:
    for line in fr:
        points = json.loads(line)['route']
        if len(points) <= 10:
            continue
        # print points
        l = LineString(json.loads(line)['route'])
        feature_list.append(
            Feature(geometry=l,
                    properties={'source': json.loads(line)['source'],
                                'destination': json.loads(line)['destination']}
                   ))
    fc = FeatureCollection(feature_list)
with open('dummy_2.geojson', 'w') as fw:
    fw.write(json.dumps(fc))
# with open('port_to_place.txt', 'r') as fr:
#    for line in fr:
#        print line
#        json.loads(line)
# break

# data = [[-8.49884, 33.24333], [-8.498674, 33.243385], [-8.498329, 33.242282], [-8.494553, 33.244388], [-8.454127, 33.246574], [-7.417327, 33.649833], [-6.921027, 33.946506], [-6.89799, 33.946712], [-6.711403, 34.02835], [-6.024201, 35.444042], [-6.028676, 35.447723], [-6.03405, 35.449821], [-6.034499, 35.450054], [-6.032969, 35.456073], [-6.036545, 35.461928]]
# print LineString(data)